# LaTeX Automation for NYCHA Waste Individual Action Plans

In [76]:
import sys
import os
import math
import subprocess
import shutil
import datetime
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from PIL import Image
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
#Set Global Vars and Options
os.chdir('/Users/kyleslugg/Documents/NYCHA/Production')
cons_tds = '073'
pd.set_option('display.max_columns', None)

In [3]:
def get_standardized_names():
    databook_names = pd.read_csv('DATA/name_tables/dev_data_book_name.csv')
    databook_names['CONS_TDS'] = databook_names['CONS_TDS'].apply(lambda x: str(int(x)).zfill(3))
    databook_names['TDS'] = databook_names['TDS'].apply(lambda x: str(int(x)).zfill(3))

    staff_names = pd.read_csv('DATA/name_tables/staff_cons_name.csv')
    staff_names['RC_Name'] = staff_names['RC Name']
    
    consolidations = {}
    developments = {}

    for row in databook_names.itertuples():
        consolidations[row.CONS_TDS] = {'name':row.CONS_NAME, 'alternates':[row.MANAGED_BY]}
        developments[row.TDS] = {'name':row.DEV_NAME, 'name_alternates':[], 'cons_tds':row.CONS_TDS}
        
    def find_closest_fuzzy_match(name, comp_df, comp_col_name, return_col_name):
        values = comp_df[comp_col_name].unique()
        comp_df_copy = pd.DataFrame(data=values, index=[i for i in range(0,len(values))], columns=[comp_col_name])

        '''
        def strip_name(x):
            string = str(x).lower()
            string = string.replace('consolidated','')
            string = string.replace('consolidation', '')
            string = string.replace('houses', '')
            return string

        comp_df_copy['partial_ratio'] = comp_df_copy[comp_col_name].apply(lambda x: fuzz.partial_ratio(strip_name(name), strip_name(x)))
        highest_match = comp_df_copy['partial_ratio'].max()

        matches = comp_df_copy.loc[comp_df_copy['partial_ratio']==highest_match, 'CONS_NAME']

        if matches.shape[0] == 1:
            return matches.iloc[0]
        else:
            print(matches)
            return 'ZZZ MULTIPLE MATCHES FOUND'

        '''
        return process.extractOne(str(name).lower(), values.tolist())[0]
    
    staff_names['NAME_MATCH'] = staff_names['RC Name'].apply(lambda x: find_closest_fuzzy_match(x, databook_names, 'CONS_NAME', 'CONS_NAME'))

    match_corrections = {'Justice Sonia Sotomayor  Consolidated': 'SOTOMAYOR HOUSES CONSOLIDATED',
                        'Murphy Consolidated': 'ZZZUNKNOWN'
                        }

    def make_corrections(row, index_col, data_col, dictionary):
        if str(row[index_col]).strip() in dictionary.keys():
            return dictionary[row[index_col]]
        else:
            return row[data_col]

    staff_names['AMENDED_MATCHES'] = staff_names.apply(lambda row: make_corrections(row, 'RC Name', 'NAME_MATCH', match_corrections), axis=1)

    staff_names = staff_names.merge(databook_names[['CONS_NAME', 'CONS_TDS']], left_on='AMENDED_MATCHES', right_on='CONS_NAME', how='left')

    for row in staff_names.itertuples():
        try:
            consolidations[row.CONS_TDS]['alternates'].append(row.RC_Name)
        except:
            print(f'TDS #{row.CONS_TDS} raised an exception.')
    
    return(consolidations, developments)


In [4]:
consolidations, developments = get_standardized_names()

counts ={}
for key, value in developments.items():
    if value['cons_tds'] not in counts.keys():
        counts[value['cons_tds']] = {'developments':[key],
                             'count':1}
    else:
        counts[value['cons_tds']]['developments'].append(key)
        counts[value['cons_tds']]['count']+=1
        


TDS #nan raised an exception.


In [5]:
count_list = [value['count'] for key, value in counts.items()]
high_count_cons = [key for key, value in counts.items() if value['count']>=8]
high_count_cons

['167', '359', '091', '530', '127']

In [6]:
def load_overview_data():#Load Data
    overview_data = pd.read_csv('DATA/overview_table_data.csv')
    overview_data['CONS_TDS'] = overview_data['CONS_TDS'].apply(lambda x: str(x).zfill(3))
    overview_data['TDS'] = overview_data['TDS'].apply(lambda x: str(x).zfill(3))
    
    return overview_data

In [7]:
overview_data = load_overview_data()
cons_list = overview_data['CONS_TDS'].unique().tolist()

## Parse and Process Text Blocks

TO COME:
- Waste Services and Assets
- Waste Distribution (top text -- bottom accounted for below)
- What Is an IAP?

In [77]:
#Lightly modified version of example found at http://etienned.github.io/posts/extract-text-from-word-docx-simply/

try:
    from xml.etree.cElementTree import XML
except ImportError:
    from xml.etree.ElementTree import XML
import zipfile


"""
Module that extract text from MS XML Word document (.docx).
(Inspired by python-docx <https://github.com/mikemaccana/python-docx>)
"""

WORD_NAMESPACE = '{http://schemas.openxmlformats.org/wordprocessingml/2006/main}'
PARA = WORD_NAMESPACE + 'p'
TEXT = WORD_NAMESPACE + 't'


def get_docx_text(path):
    """
    Take the path of a docx file as argument, return the text in unicode.
    """
    document = zipfile.ZipFile(path)
    try:
        xml_content = document.read('word/document.xml')
    except:
        xml_content = document.read('word/document2.xml')
        
    document.close()
    tree = XML(xml_content)

    paragraphs = []
    for paragraph in tree.getiterator(PARA):
        texts = [node.text
                 for node in paragraph.getiterator(TEXT)
                 if node.text]
        if texts:
            paragraphs.append(''.join(texts))

    return '\n\n'.join(paragraphs)

In [8]:
#Character Substitutions for LaTeX -- set and define "clean" method
def clean_text(text):
    substitutions = {'“':"``",
                '”': "''",
                '’':"'",
                ' ':' ',
                '–':'--',
                ' ':' ',
                '\xa0':' '}
    
    for key, value in substitutions.items():
        text = text.replace(key, value)
        
    return text

#### Overview Text

In [68]:
def make_overview_text(cons_tds):
    header = re.compile(r'((\w*\s)*(Overview)):?')
    
    overview_text = get_docx_text(f'TEXT/overview_text/{cons_tds}_Overview.docx')
    if len(header.findall(overview_text)) == 0:
            overview_text = overview_text
    else:
        try:
            overview_text = overview_text.replace(header.findall(overview_text)[0]+'\n','')
        except:
            overview_text = overview_text.replace(header.findall(overview_text)[0][0]+'\n','')
    
    overview_text = clean_text(overview_text)
    
    with open(f'TEXT/overview_text/{cons_tds}_overview.tex', 'w') as file_handle:
        file_handle.write(overview_text)
    
    pass

In [71]:
for tds in consolidations.keys():
    try:
        make_overview_text(tds)
    #except KeyError:
        #pass
    #except FileNotFoundError:
        #pass
    except:
        print(f"{tds} raised error")

073 raised error
031 raised error
210 raised error
016 raised error
128 raised error


#### Analysis Text

In [95]:
def make_analysis_text(cons_tds):
    analysis_text = get_docx_text(f'TEXT/analysis_text/{cons_tds}_Analysis.docx')

    header = re.compile(r'([\w\s]*:)')
    
    analysis_text = clean_text(analysis_text)

    section_headings = {'Inspection and Collection Requirement':['Inspection and Collection Requirements',
                                                                 'Inspection and Collection Requirement',
                                                                 'Collection and Inspection Requirements',
                                                                'Collection and Inspection Requirement'],
                        'Removal or Storage Requirement':['Removal or Storage Requirements',
                                                          'Removal or Storage Requirement',
                                                          'Removal and Storage Requirements',
                                                         'Removal and Storage Requirement',
                                                         'Storage or Removal Requirement',
                                                          'Storage and Removal Requirements',
                                                         'Storage and Removal Requirement',
                                                         'Removal or Storage Requirement '],
                       'Additional Context':['Additional Context']}
    
    for heading, variants in section_headings.items():
        for variant in variants:
            if variant in analysis_text:
                analysis_text = analysis_text.replace(variant, r'\textbf{%s}' % (heading))
                break

    if len(header.findall(analysis_text)) == 0:
        pass
    else:
        analysis_text = analysis_text.replace(header.findall(analysis_text)[0]+'\n','')

    latex_block = analysis_text

    with open(f'TEXT/analysis_text/{cons_tds}_analysis.tex', 'w') as file_handle:
        file_handle.write(latex_block)
        
    pass

In [96]:
for tds in consolidations.keys():
    try:
        make_analysis_text(tds)
    #except FileNotFoundError:
        #pass
    except:
        print(f"{tds} raised error")

167 raised error
097 raised error
359 raised error
073 raised error
153 raised error
210 raised error
091 raised error
165 raised error
337 raised error
056 raised error
065 raised error
252 raised error
016 raised error
530 raised error
086 raised error
113 raised error
075 raised error
134 raised error
170 raised error
351 raised error
127 raised error
041 raised error
169 raised error
044 raised error
341 raised error
010 raised error
081 raised error
021 raised error
083 raised error
112 raised error
093 raised error
035 raised error
005 raised error
055 raised error
377 raised error
128 raised error


In [ ]:
'''By understanding how much waste is generated at each consolidation, planners and managers
can better determine how well current assets and services serve current needs, and what additional 
elements are necessary in order for each consolidation to operate as efficiently as possible. 

%s has (%s) 30-CY external compactors, amounting to %s ft2 of storage space for garbage. 
Given the rate at which waste is produced at NYCHA properties, these containers will fill
up in about (3) days at the Sumner Consolidation. The average weight of the containers at
capacity should be about (3) tons.'''

## Select and Prepare Maps

In [ ]:
# Set asset map path
asset_map_path = f"MAPS/asset_maps/{cons_tds}_asset_map.png"

In [ ]:
# Split context map into two pages
def process_context_map(cons_tds):
    image = Image.open(f'MAPS/context_maps/{cons_tds}_context_map.png')
    width, height = image.size

    bb1 = (0,0,width/2,height)
    bb2 = (width/2, 0, width, height)

    img_1 = image.crop(bb1)
    img_2 = image.crop(bb2)

    img_1.save(f'MAPS/context_maps/{cons_tds}_context_1.png', format="PNG")
    img_2.save(f'MAPS/context_maps/{cons_tds}_context_2.png', format="PNG")
    
    pass

In [ ]:
for tds in consolidations.keys():
    try:
        process_context_map(tds)
    except:
        pass

## Produce Tables

TO COME:
- Consolidation Assets

#### Make Overview Table

In [33]:
def make_overview_table(cons_tds, overview_data=overview_data):
    
    cons_data = overview_data.loc[overview_data['CONS_TDS']== cons_tds]
    
    overview_table = ''

    overview_frame = r'''
    \resizebox{\textwidth}{!}{
    \begin{tabular}{l|c|c|c|c|}
    \cline{2-5}
                                                                           & \cellcolor{ccteal}{\color[HTML]{FFFFFF} TDS \#} & \cellcolor{ccteal}{\color[HTML]{FFFFFF} Total Households} & \cellcolor{ccteal}{\color[HTML]{FFFFFF} Official Population} & \cellcolor{ccteal}{\color[HTML]{FFFFFF} Average Family Size} \\ \hline

    '''

    development_template = r'''\multicolumn{1}{|l|}{\cellcolor{ccteallight}%s}        & %s                                                   & %s                                                           & %s                                                                & %s                                                                \\ \hline'''


    overview_table += overview_frame

    for row in cons_data.itertuples():
        dev_name = row.DEV_NAME.title()
        dev_tds = row.TDS
        total_hhs = row.TOTAL_HH
        official_population = row.TOTAL_POP
        avg_family_size = row.AVG_FAMILY_SIZE

        overview_table += development_template % (dev_name, dev_tds, total_hhs, official_population, avg_family_size)

    overview_table += r'''
    \end{tabular}
    }
    '''
    
    with open(f'TABLES/overview_table/{cons_tds}_overview_table.tex', 'w') as file_handle:
        file_handle.write(overview_table)
    

In [34]:
for tds in consolidations.keys():
    make_overview_table(tds)

#### Typology Table

In [35]:
def load_typology_data():
    #Cleaning and Shaping Data
    typ_1 = pd.read_csv('DATA/typologies_1.csv')
    typ_2 = pd.read_csv('DATA/typologies_2.csv')

    typ_1.columns = ['CONS_NAME', 'DEV_NAME', 'TDS', 'TYPOLOGY']
    typ_2.columns = ['CONS_NAME', 'CONS_TDS', 'DEV_NAME', 'TDS', 'METHOD', 
                     'CONSTRUCTION_DATE', 'BLDG_AGE', 'STORIES', 'BLDG_COVERAGE_SQFT', 'OPEN_SPACE_RATIO', 'SCATTERED_SITE_FLAG']

    def make_dates(date_col):
        date = str(date_col).split('/')
        try:
            if int(date[2]) > 18:
                return datetime.date(int(f'19{date[2]}'), int(date[0]), int(date[1]))
            else:
                return datetime.date(int(f'20{date[2]}'), int(date[0]), int(date[1]))
        except IndexError:
            return datetime.date(1900,1,1)

    typ_2['CONSTRUCTION_DATE'] = typ_2['CONSTRUCTION_DATE'].apply(lambda x: make_dates(x))
    typ_2['SCATTERED_SITE_FLAG'] = typ_2['SCATTERED_SITE_FLAG'].apply(lambda x: x == 'YES')
    typ_2.loc[typ_2['SCATTERED_SITE_FLAG']=='YES','SCATTERED_SITE_FLAG'] = 1

    typology = typ_1.merge(typ_2[['CONS_TDS', 'TDS', 'METHOD',
                                 'CONSTRUCTION_DATE', 'BLDG_AGE', 
                                 'STORIES', 'BLDG_COVERAGE_SQFT', 
                                 'OPEN_SPACE_RATIO', 'SCATTERED_SITE_FLAG']], how='left', on='TDS')

    typology['CONS_TDS'] = typology['CONS_TDS'].apply(lambda x: str(int(x)).zfill(3))
    typology['PREWAR'] = typology['CONSTRUCTION_DATE'].apply(lambda x: x < datetime.date(1945,1,1))

    #Adding Typology Icons

    typ_icons = [r'\rootpath/IMAGES/typology_earlytower.png', r'\rootpath/IMAGES/typology_towerpark.png', r'\rootpath/IMAGES/typology_prewar.png', r'\rootpath/IMAGES/typology_scatteredsite.png']
    typ_dict = {}
    [typ_dict.setdefault(key, '') for key in typology['TYPOLOGY'].unique().tolist()]

    typ_dict['1 - High-rise in the park'] = typ_icons[1]
    typ_dict['2 - Mid-rise in the park'] = typ_icons[1]
    typ_dict['3 - Low-rise in the park'] = typ_icons[0]
    typ_dict['4 - Context Towers'] = typ_icons[3]
    typ_dict['5 - Context Mid-rises'] = typ_icons[2]
    typ_dict['6 - Walkups & Brownstones'] = typ_icons[2]

    typ_header = re.compile(r'\d\s-\s')

    typology['TYP_NAME'] = typology['TYPOLOGY'].apply(lambda x: typ_header.sub('', str(x)))
    typology['IMAGE_PATH'] = typology['TYPOLOGY'].apply(lambda x: typ_dict[x])
    
    return typology

In [36]:
typology = load_typology_data()

/Users/kyleslugg/opt/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [37]:
def make_typology_table_block(cons_tds, typ_data):
    cons_data = typ_data[typ_data['CONS_TDS'] == cons_tds]
    num_devs = cons_data.shape[0]
    
    if num_devs < 5:
        block_1 = cons_data
    
    elif num_devs >=5 and num_devs < 7:
        block_1 = cons_data.iloc[0:3]
        block_2 = cons_data.iloc[3:]
    
    else:
        block_1 = cons_data.iloc[0:4]
        block_2 = cons_data.iloc[4:]
    
    len_1 = block_1.shape[0]
    
    try:
        len_2 = block_2.shape[0]
    except:
        len_2 = 0
    
    headers = {1:r"\begin{tabular}{m{1.5in} m{2in}"+'\n',
              2:r"\begin{tabular}{m{1.25in} m{2in} m{.1in} m{1.25in} m{2in}}"+'\n',
              3:r"\begin{tabular}{m{1.25in} m{1.5in} m{.2in} m{1.25in} m{1.5in} m{.2in} m{1.25in} m{1.5in}}"+'\n',
              4:r"\begin{tabular}{m{1.25in} m{1.25in} m{.2in} m{1.25in} m{1.25in} m{.2in} m{1.25in} m{1.25in} m{.2in} m{1.25in} m{1.25in}}"+'\n'}
         
    lines = {1:r'''\textbf{%s:} {%s} & \includegraphics[height=2in]{%s}'''+'\n'+r'\end{tabular}',
            2:r'''\textbf{%s:} {%s} & \includegraphics[height=2in]{%s} & & \textbf{%s:} {%s} & \includegraphics[height=2in]{%s}'''+'\n'+r'\end{tabular}',
            3:r'''\textbf{%s:} {%s} & \includegraphics[height=1.5in]{%s} & & \textbf{%s:} {%s} & \includegraphics[height=1.5in]{%s} & & \textbf{%s:} {%s} & \includegraphics[height=1.5in]{%s}'''+'\n'+r'\end{tabular}',
            4:r'''\textbf{%s:} {%s} & \includegraphics[height=1.5in]{%s} & & \textbf{%s:} {%s} & \includegraphics[height=1.5in]{%s} & & \textbf{%s:} {%s} & \includegraphics[height=1.5in]{%s}& & \textbf{%s:} {%s} & \includegraphics[height=1.5in]{%s}'''+'\n'+r'\end{tabular}'}
    
    
    data_1 = []
    data_2 = []
    
    for row in block_1.itertuples():
        data_1.append(str(row.DEV_NAME).title())
        data_1.append(str(row.TYP_NAME).replace('&', '\&'))
        data_1.append(row.IMAGE_PATH)
    
    if len_2 > 0:
        for row in block_2.itertuples():
            data_2.append(str(row.DEV_NAME.title()))
            data_2.append(str(row.TYP_NAME).replace('&', '\&'))
            data_2.append(row.IMAGE_PATH)
    
    # Assembling Nested Tables
    latex_block = ''
    latex_block += r'''\begin{table}[H]
    \resizebox{\textwidth}{!}{
    \begin{tabular}{c}
    '''
    
    latex_block += headers[len_1]
    latex_block += lines[len_1] % tuple(data_1)
    
    if len_2 > 0:
        latex_block += r'''\\
        '''
        latex_block += headers[len_2]
        latex_block += lines[len_2] % tuple(data_2)
    
    latex_block += r'''\end{tabular}}
    \end{table}'''
    
    with open(f'TABLES/typology_table/{cons_tds}_typology.tex', 'w') as file_handle:
        file_handle.write(latex_block)

In [38]:
typology = load_typology_data()

for tds in consolidations.keys():
    try:
        make_typology_table_block(tds, typology)
    except:
        print(f'{tds} raised an exception.')

210 raised an exception.
091 raised an exception.
128 raised an exception.


In [39]:
problem_cons_tds = {'091':'Baisley Park. Isolate important developments.',
                   '359':'Skip for now.'}


#### Waste Services and Assets

In [40]:
def load_wsa_data():
    wsa_data = pd.read_csv('DATA/WASTE_SERVICES_ASSETS.csv')
    wsa_data['TDS'] = wsa_data['DEV_TDS'].apply(lambda x: str(x).zfill(3))
    wsa_data['INT_COMP_DATE'] = pd.to_datetime(wsa_data['INT_COMP_INSTALL_DATE'], errors='ignore')
    
    return wsa_data

In [41]:
def make_waste_services_table(cons_tds, wsa_data, counts_dict=counts):
    dev_list = counts_dict[cons_tds]['developments']
    cons_data = wsa_data.query(f"TDS in {dev_list}")
    num_devs = counts_dict[cons_tds]['count']
    
    def make_waste_services_block(num_cols, block_data):
    
        col_format = r'X|'
        header = r'\begin{tabularx}{\textwidth}{V{1.5in}|'+col_format*(num_cols)+r'''}
    \cline{2-4}
                                                                                       '''+r'& \cellcolor{ccorange}{\color[HTML]{FFFFFF} %s}'*num_cols+r' \\ \hline'+'\n'
        hh_waste_line = r'\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}Household Waste (DSNY)}               '+r'& %s'*num_cols+r'\\ \hline'+'\n'
        bulk_waste_line = r'\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}Bulk Waste}                  '+r'& %s'*num_cols+r' \\ \hline'+'\n'
        norm_recycling_line = r'\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s}                   '+r'& DSNY Curb Setout'*num_cols + r'\\ \hline'+'\n'
        special_recycling_line = r'\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s}                   '+r'& %s'*num_cols +r'\\ \hline' + '\n'
        
        latex_block = r''''''
        latex_block += header % tuple(block_data['DEV_NAME'].apply(lambda x: str(x).title()).tolist())
        
        
        hh_waste_data = []
        bulk_waste_data = []
        ewaste_data = []
        textiles_data = []
        
        for dev in block_data.itertuples():
            if dev.CURBSIDE == 1:
                hh_waste_data.append('Curbside Pickup')
            elif dev.SHARE == 1:
                hh_waste_data.append(f'Transfer to {str(dev.SHARE_SITE).title()}')
            else:
                if (dev.EXT_COMP_BE == 1) and (dev.COMPACTOR_YARDS == 1):
                    hh_waste_data.append(f'{int(dev.EXT_COMP_BE)} exterior compactor in {int(dev.COMPACTOR_YARDS)} waste yard')
                elif (dev.COMPACTOR_YARDS == 1):
                    hh_waste_data.append(f'{int(dev.EXT_COMP_BE)} exterior compactors in {int(dev.COMPACTOR_YARDS)} waste yard')
                else:
                    hh_waste_data.append(f'{int(dev.EXT_COMP_BE)} exterior compactors in {int(dev.COMPACTOR_YARDS)} waste yards')
                
            if pd.isna(dev.BULK_HAULER):
                if int(dev.BULK_SITES) == 0:
                    bulk_waste_data.append("Transferred for Pickup")
                elif int(dev.BULK_SITES) == 1:
                    bulk_waste_data.append("One Bulk Drop Site; Transferred for Pickup")
                else:
                    bulk_waste_data.append(f"{dev.BULK_SITES} Bulk Drop Sites; Transferred for Pickup")
            else:
                if int(dev.BULK_SITES) == 1:
                    bulk_waste_data.append(f"One Bulk Drop Site; Picked up by {dev.BULK_HAULER}")
                elif int(dev.BULK_SITES) > 1:
                    bulk_waste_data.append(f"{dev.BULK_SITES} Bulk Drop Sites; Picked up by {dev.BULK_HAULER}")
                else:
                    bulk_waste_data.append(f"Picked up by {dev.BULK_HAULER}")
            
            if dev.ECYCLE == 1:
                ewaste_data.append('Previously available through ECycle')
            else:
                ewaste_data.append('N/A')
            
            if dev.REFASHION == 1:
                textiles_data.append('Previously available through Refashion')
            else:
                textiles_data.append('N/A')
        
        latex_block += hh_waste_line % tuple(hh_waste_data)
        latex_block += bulk_waste_line % tuple(bulk_waste_data)
        latex_block += norm_recycling_line % 'Recycling: Paper and Cardboard'
        latex_block += norm_recycling_line % 'Recycling: Metal, Glass, and Plastic'
        latex_block += special_recycling_line % tuple(['Recycling: Mattresses']+['N/A' for i in range(0, num_cols)])
        latex_block += special_recycling_line % tuple(['Recycling: E-Waste']+ewaste_data)
        latex_block += special_recycling_line % tuple(['Recycling: Textiles']+textiles_data)
        latex_block += r'\end{tabularx}'
        
        return latex_block
    
    if num_devs <= 4:
        num_cols = num_devs
        block_data = cons_data
        
        with open(f'TABLES/waste_services/{cons_tds}_waste_services.tex', 'w') as file_handle:
            file_handle.write(make_waste_services_block(num_cols, block_data))
        
    elif num_devs > 4:
        num_cols_1 = math.ceil(num_devs/2)
        num_cols_2 = (num_devs-num_cols_1)
        block_data_1 = cons_data.iloc[0:num_cols_1]
        block_data_2 = cons_data.iloc[num_cols_1:]
        
        with open(f'TABLES/waste_services/{cons_tds}_waste_services_1.tex', 'w') as file_handle:
            file_handle.write(make_waste_services_block(num_cols_1, block_data_1))
            
        with open(f'TABLES/waste_services/{cons_tds}_waste_services_2.tex', 'w') as file_handle:
            file_handle.write(make_waste_services_block(num_cols_2, block_data_2))
    
    pass

In [42]:
wsa_data = load_wsa_data()

for tds in consolidations.keys():
    make_waste_services_table(tds, wsa_data)

In [43]:
def make_waste_assets_table(cons_tds, wsa_data, counts_dict=counts):
    dev_list = counts_dict[cons_tds]['developments']
    cons_data = wsa_data.query(f"TDS in {dev_list}")
    num_devs = counts_dict[cons_tds]['count']
    
    header = r'''
    \begin{tabular}{V{.25\columnwidth}|V{.15\columnwidth}|V{.15\columnwidth}|V{.25\columnwidth}|V{.15\columnwidth}|}
\cline{2-5}
                                                                                              & \cellcolor{ccorangelight}{\color[HTML]{000000} Internal Compactors} & \cellcolor{ccorangelight}{\color[HTML]{000000} External Compactors} & \cellcolor{ccorangelight}{\color[HTML]{000000} Other External Assets}   & \cellcolor{ccorangelight}{\color[HTML]{000000} Recycling Bins\tnote{1}} \\ \hline'''+'\n'
    line_format = r'\multicolumn{1}{|V{.25\columnwidth}|}{\cellcolor{ccorange}{\color[HTML]{FFFFFF} %s}}        & %s                                                & %s                                                                  & %s & %s                                                            \\ \hline'+'\n'
    
    latex_block = r''''''
    latex_block += header
    
    for dev in cons_data.itertuples():
        line_data = []
        line_data.append(str(dev.DEV_NAME).title())
        
        if (dev.INT_COMP == 0):
            int_comp_string = '0'
        elif pd.isna(dev.INT_COMP_DATE):
            int_comp_string = str(int(dev.INT_COMP))
        else:
            int_comp_string = f'{str(int(dev.INT_COMP))}; last replaced {str(dev.INT_COMP_DATE.year)}'
        
        line_data.append(int_comp_string)
        line_data.append(str(int(dev.EXT_COMP_BE)))
        
        #if (dev.BULK_CRUSHERS == 0) and (dev.BALERS == 0)... REDO THIS ONCE DATA ARE COMPLETE
        line_data.append('PLACEHOLDER UNTIL DATA ARE COMPLETE')
        line_data.append(str(int(dev.RECYCLING_BINS)))
        
        latex_block += line_format % tuple(line_data)
    
    latex_block += r'\end{tabular}'
    
    with open(f'TABLES/waste_assets/{cons_tds}_waste_assets.tex', 'w') as file_handle:
        file_handle.write(latex_block)
    
    pass
    

In [44]:
wsa_data = load_wsa_data()
for tds in consolidations.keys():
    make_waste_assets_table(tds, wsa_data)

#### Waste Calculator

In [45]:
def add_waste_cols(overview_data):
    conversion_factors = {'units_to_tons_day': 0.0025,
                         'cy_per_ton': {'trash': 21.05,
                                        'trash_actual': 0,
                                       'MGP': 18.02,
                                       'cardboard': 26.67,
                                       'paper': 6.19,
                                       'organics': 4.32,
                                       'ewaste': 5.65,
                                       'textiles': 13.33},
                         'gallons_per_cy': 201.974,
                         'gallons_per_64gal': 64,
                         'gallons_per_40lb_bag': 44,
                         'cy_per_44gal_bag':0.174,
                         'cy_per_cardboard_bale':0.193}

    waste_percentages = {'trash': .26,
                         'trash_actual':.894,
                        'MGP': .19,
                        'cardboard': .07,
                        'paper': .07,
                        'organics':.32,
                        'ewaste': .01,
                        'textiles': .08}

    capture_rates = {'trash_primary': .75,
                    'trash_secondary': .25,
                    'mgp': .30,
                    'cardboard': .50,
                    'paper': .20}

    overview_data['WASTE_TONS_DAY'] = overview_data['CURRENT_APTS'].apply(lambda x: x * conversion_factors['units_to_tons_day'])

    for key, value in waste_percentages.items():
        overview_data[f'{key.upper()}_CY'] = overview_data['WASTE_TONS_DAY'].apply(lambda x: x * value * conversion_factors['cy_per_ton'][key])
        overview_data[f'{key.upper()}_TONS'] = overview_data['WASTE_TONS_DAY'].apply(lambda x: x * value)
    
    overview_data['TRASH_ACTUAL_CY'] = (overview_data['TRASH_CY']+
                                           overview_data['MGP_CY']+
                                           overview_data['CARDBOARD_CY']+
                                           overview_data['PAPER_CY']+
                                           overview_data['ORGANICS_CY']+
                                           overview_data['EWASTE_CY']+
                                           overview_data['TEXTILES_CY'])-(overview_data['MGP_CY']*capture_rates['mgp']+
                                                                         overview_data['CARDBOARD_CY']*capture_rates['cardboard']+
                                                                         overview_data['PAPER_CY']*capture_rates['paper'])

    overview_data['TRASH_CHUTE_CY'] = overview_data['TRASH_ACTUAL_CY']*capture_rates['trash_primary']
    overview_data['TRASH_CHUTE_TONS'] = overview_data['TRASH_ACTUAL_TONS']*capture_rates['trash_primary']
    overview_data['TRASH_CHUTE_SAUSAGE'] = ((overview_data['TRASH_CHUTE_CY'])/conversion_factors['cy_per_ton']['trash'])*(2000/40)
    overview_data['TRASH_DROP_CY'] = overview_data['TRASH_ACTUAL_CY']*capture_rates['trash_secondary']
    overview_data['TRASH_DROP_TONS'] = overview_data['TRASH_ACTUAL_TONS']*capture_rates['trash_secondary']
    overview_data['TRASH_DROP_BINS'] = overview_data['TRASH_DROP_CY']*conversion_factors['gallons_per_cy']/64
    overview_data['CAPTURED_MGP_TONS_WEEK'] = overview_data['MGP_TONS']*capture_rates['mgp']*7
    overview_data['CAPTURED_CARDBOARD_TONS_WEEK'] = overview_data['CARDBOARD_TONS']*capture_rates['cardboard']*7
    overview_data['CAPTURED_PAPER_TONS_WEEK'] = overview_data['PAPER_TONS']*capture_rates['paper']*7
    overview_data['MGP_BAGS_WEEK'] = overview_data['MGP_CY']*capture_rates['mgp']*7/conversion_factors['cy_per_44gal_bag']
    overview_data['PAPER_BAGS_WEEK'] = overview_data['PAPER_CY']*capture_rates['paper']*7/conversion_factors['cy_per_44gal_bag']
    overview_data['CARDBOARD_BALES_WEEK'] = overview_data['CARDBOARD_CY']*capture_rates['cardboard']*7/conversion_factors['cy_per_cardboard_bale']
    
    return overview_data

In [46]:
def make_waste_distribution_table(cons_tds, overview_data):
    cons_data = overview_data[overview_data['CONS_TDS'] == cons_tds]
    num_devs = cons_data.shape[0]
    
    if num_devs == 1:
        num_cols = num_devs
    elif num_devs > 4:
        num_cols_1 = math.ceil(num_devs/2)
        num_cols_2 = (num_devs-num_cols_1)+1
    else:
        num_cols = num_devs+1
    
    if num_devs != 1:
        cons_data.loc['Total']= cons_data.sum(numeric_only=True, axis=0)
        cons_data.loc['Total','DEV_NAME'] = 'Total'
    
    def make_waste_distribution_table_block(cons_data, num_cols):
        dev_col_format = r'X|'

        opening = r'''
        \begin{tabularx}{\textwidth}{V{1.5in}|%s}
        \cline{2-%s}
        ''' % (dev_col_format*num_cols, (num_cols+1))

        top_row = r'''
                                                                       '''+(r"& \multicolumn{1}{V{1.5in}|}{\cellcolor{ccorange}%s}"*(num_cols))+r"\tnhl"+'\n'

        standard_row = r"\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s}                 "+(r"& %s                                    ")*num_cols+r"\tnhl"+'\n'

        captured_row = r"\multicolumn{1}{|Y{1.5in}|}{\cellcolor{ccorangelight}Captured / Week (tons)\tnote{4}}                        "+(r"& %s                                    ")*num_cols+r"\tnhl"+'\n'

        chute_row = r"\multicolumn{1}{|Y{1.5in}|}{\cellcolor{ccorangelight}Trash Chutes\tnote{2}}                 "+(r"& %s tons or (%s) 40 lbs. sausage bags      "*num_cols)+r"\tnhl"+'\n'

        dropsite_row = r"\multicolumn{1}{|Y{1.5in}|}{\cellcolor{ccorangelight}Drop Sites\tnote{3}}                 "+(r"& %s tons or (%s) 64-gallon bins      "*num_cols)+r"\tnhl"+'\n'

        OET_row = r"\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s / Day (CY)\tnote{5}}              "+(r"& %s                                    "*num_cols)+r"\tnhl"+'\n'

        recycling_row = r"\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s}                 "+(r"& %s tons or (%s) 44-gallon bags                                   ")*num_cols+r"\tnhl"+'\n'

        cardboard_row = r"\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s}                 "+(r"& %s tons or (%s) bales                                   ")*num_cols+r"\tnhl"+'\n'


        def make_trash_text(row, text_var, cy_col, other_col):
            text_var.append(round(row[cy_col],2))
            text_var.append(round(row[other_col], 2))
            pass

        latex_block = opening
        latex_block += top_row % tuple(cons_data['DEV_NAME'].apply(lambda x: str(x).title()).tolist())
        latex_block += standard_row % tuple([r"Waste Generated / Day (Tons)\tnote{1}"]+[round(item, 2) for item in cons_data['WASTE_TONS_DAY'].tolist()])
        latex_block += standard_row % tuple([r"Trash / Day (tons)\tnote{2}"]+cons_data['TRASH_ACTUAL_TONS'].apply(lambda x: str(round(x,2))).tolist())

        trash_chute_text = []
        dropsite_text = []

        cons_data.apply(lambda row: make_trash_text(row, trash_chute_text, 'TRASH_CHUTE_TONS', 'TRASH_CHUTE_SAUSAGE'), axis=1)
        cons_data.apply(lambda row: make_trash_text(row, dropsite_text, 'TRASH_DROP_TONS', 'TRASH_DROP_BINS'), axis=1)

        latex_block += chute_row % tuple(trash_chute_text)
        latex_block += dropsite_row % tuple(dropsite_text)

        latex_block += r"\end{tabularx}\bigskip"

        latex_block += opening
        latex_block += top_row % tuple(cons_data['DEV_NAME'].apply(lambda x: str(x).title()).tolist())

        mgp_text = []
        cardboard_text= []
        paper_text = []

        cons_data.apply(lambda row: make_trash_text(row, mgp_text, 'CAPTURED_MGP_TONS_WEEK', 'MGP_BAGS_WEEK'), axis=1)

        cons_data.apply(lambda row: make_trash_text(row, cardboard_text, 'CAPTURED_CARDBOARD_TONS_WEEK', 'CARDBOARD_BALES_WEEK'), axis=1)

        cons_data.apply(lambda row: make_trash_text(row, paper_text, 'CAPTURED_PAPER_TONS_WEEK', 'PAPER_BAGS_WEEK'), axis=1)



        latex_block += recycling_row % tuple([r"Metal, Glass, Plastic Captured / Week (tons)"]+mgp_text)
        #latex_block += captured_row % tuple(cons_data['CAPTURED_MGP_CY'].apply(lambda x: str(round(x,2))).tolist())
        latex_block += cardboard_row % tuple([r"Cardboard Captured / Week (tons)"]+cardboard_text)
        #latex_block += captured_row % tuple(cons_data['CAPTURED_CARDBOARD_CY'].apply(lambda x: str(round(x,2))).tolist())
        latex_block += recycling_row % tuple([r"Paper Captured / Week (tons)"]+paper_text)
        #latex_block += captured_row % tuple(cons_data['CAPTURED_PAPER_CY'].apply(lambda x: str(round(x,2))).tolist())

        #latex_block += OET_row % tuple(['Organics']+cons_data['ORGANICS_CY'].apply(lambda x: str(round(x,2))).tolist())
        #latex_block += OET_row % tuple(['E-Waste']+cons_data['EWASTE_CY'].apply(lambda x: str(round(x,2))).tolist())
        #latex_block += OET_row % tuple(['Textiles']+cons_data['TEXTILES_CY'].apply(lambda x: str(round(x,2))).tolist())

        latex_block += r"\end{tabularx}"

        return latex_block
    
    if num_devs<= 4:
        latex_block = make_waste_distribution_table_block(cons_data, num_cols)

        with open(f'TABLES/waste_distribution_table/{cons_tds}_wd_table.tex', 'w') as file_handle:
            file_handle.write(latex_block)
    
    else:
        latex_block_1 = make_waste_distribution_table_block(cons_data.iloc[0:num_cols_1], num_cols_1)
        latex_block_2 = make_waste_distribution_table_block(cons_data.iloc[num_cols_1:], num_cols_2)
        
        with open(f'TABLES/waste_distribution_table/{cons_tds}_wd_table_1.tex', 'w') as file_handle:
            file_handle.write(latex_block_1)
        with open(f'TABLES/waste_distribution_table/{cons_tds}_wd_table_2.tex', 'w') as file_handle:
            file_handle.write(latex_block_2)

    text_block = r''''''

    text_line_multi = r"\bf{%s}: This development has %s apartment units and %s stairhalls.\\"

    text_line_singular = r"\bf{%s}: This development has %s apartment units and one stairhall.\\"

    for row in cons_data.itertuples():

        if int(row.STAIRHALLS) == 1:
            text_block += text_line_singular % (row.DEV_NAME.title(), int(row.CURRENT_APTS))
        else:
            text_block += text_line_multi % (row.DEV_NAME.title(), int(row.CURRENT_APTS), int(row.STAIRHALLS))


    with open(f'TEXT/waste_distribution_bottom/{cons_tds}_wd_bottom.tex', 'w') as file_handle:
        file_handle.write(text_block)

    

In [47]:
overview_data = add_waste_cols(load_overview_data())
for tds in consolidations.keys():
    try:
        make_waste_distribution_table(tds, overview_data)
    except:
        print(f'Exception raised by {tds}')

/Users/kyleslugg/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/kyleslugg/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Exception raised by 091


#### Make Capital Improvements Table

In [48]:
def load_asset_data():
    asset_data = {'fwd': ['In-Sink Food Grinders', pd.read_csv('DATA/capital_fwd.csv')],
                  'ehd': ['Enlarged Hopper Doors', pd.read_csv('DATA/capital_ehd.csv')],
                  'int_compactor':['Interior Compactor Replacement', pd.read_csv('DATA/capital_intcom.csv')],
                  'wasteyard':['Waste Yard Redesign', pd.read_csv('DATA/capital_wasteyard.csv')]}

    for value in asset_data.values():
        value[1].columns = [item.strip() for item in value[1].columns]

    asset_data['wasteyard'][1]['ESTIMATE'] = asset_data['wasteyard'][1]['TOT_EST']
    asset_data['wasteyard'][1]['COST'] = np.nan
    
    def year_to_string(year):
        if pd.isna(year):
            return 'N/A'
        else:
            if int(year) <= 2022:
                return str(int(year))
            elif (int(year) > 2022) & (int(year) <= 2025):
                return '2023-2025'
            elif (int(year)>2025) and (int(year)<=2030):
                return '2026-2030'
            else:
                return 'After 2030'
    
    asset_data['fwd'][1]['_YEAR'] = asset_data['fwd'][1]['EST_YEAR'].apply(lambda x: year_to_string(x))
    asset_data['ehd'][1]['_YEAR'] = asset_data['fwd'][1]['CYEAR'].apply(lambda x: year_to_string(x))
    asset_data['int_compactor'][1]['_YEAR'] = asset_data['int_compactor'][1]['CYEAR'].apply(lambda x: year_to_string(x))
    asset_data['wasteyard'][1]['_YEAR'] = asset_data['wasteyard'][1]['CONS_CYEAR'].apply(lambda x: year_to_string(x))
    return asset_data

In [49]:
def make_capital_table(cons_tds, asset_data, overview_data=overview_data):
    cons_data = overview_data[overview_data['CONS_TDS'] == cons_tds]
    num_devs = cons_data.shape[0]

    def make_capital_table_block(block_data, num_devs):
        dev_col_format = r'X|'
        header = r'''
        \begin{tabularx}{\textwidth}{r|%s}
        \cline{2-%s}
        ''' % ((dev_col_format*num_devs), num_devs)

        top_row = r"\multicolumn{1}{l|}{}                                                        "+r"& \cellcolor{ccorange}{\color[HTML]{FFFFFF}%s} "*num_devs+r"\\ \hline"+"\n"

        project_block = r"\multicolumn{1}{|V{.2\columnwidth}|}{\cellcolor{ccorangelight}%s}          "+(r"&                                                                  "*num_devs)+r"\\"+r'''
        \multicolumn{1}{|r|}{\cellcolor{ccorangelight}\textit{Status}}                '''+(r"& %s                                                         "*num_devs)+r'''\\
        \multicolumn{1}{|r|}{\cellcolor{ccorangelight}\textit{%s}}                  '''+("& %s                                                     "*num_devs)+r"\\ \hline"+"\n"

        devs = block_data['DEV_NAME'].apply(lambda x: str(x).upper()).tolist()
        devs_title = block_data['DEV_NAME'].apply(lambda x: str(x).title()).tolist()
        latex_block = ''
        latex_block += header
        latex_block += top_row % tuple(block_data['DEV_NAME'].apply(lambda x: str(x).title()).tolist())

        for asset in asset_data.keys():
            asset_df = asset_data[asset][1]
            #print(asset_data[asset][0])
            #print(devs)
            #print(asset_df['DEVELOPMENT'].tolist())
            if any((dev in asset_df['DEVELOPMENT'].tolist()) for dev in devs):
                status_list = []
                year_list = []

                for dev in devs:
                    if dev in asset_df['DEVELOPMENT'].tolist():
                        #print(dev)
                        if pd.isna(asset_df.loc[asset_df['DEVELOPMENT']== dev,'STATUS'].iloc[0]):
                            status_list.append('Not Yet Scheduled')
                        else:
                            status_list.append(str(asset_df.loc[asset_df['DEVELOPMENT']== dev, 'STATUS'].iloc[0]).title())

                        #print(asset_df.loc[asset_df['DEVELOPMENT']== dev, 'STATUS'])
                        #print(asset_df.loc[asset_df['DEVELOPMENT']== dev, 'COST'])
                        #try:
                        year_list.append(str(asset_df.loc[asset_df['DEVELOPMENT']== dev,'_YEAR'].iloc[0]))
                    #except:
                            #year_list.append('TBD')

                    else:
                        status_list.append('N/A')
                        year_list.append(' ')

                asset_block = project_block % tuple([asset_data[asset][0]]+status_list+['Year Planned']+year_list)

                latex_block += asset_block

        latex_block += r"\end{tabularx}"

        return latex_block
    
    
    if num_devs <= 4:
        num_cols = num_devs
        block_data = cons_data
        
        with open(f"TABLES/capital_projects_table/{cons_tds}_capital_projects.tex", 'w') as file_handle:
            file_handle.write(make_capital_table_block(block_data, num_cols))
        
    elif num_devs > 4:
        num_cols_1 = math.ceil(num_devs/2)
        num_cols_2 = (num_devs-num_cols_1)
        block_data_1 = cons_data.iloc[0:num_cols_1]
        block_data_2 = cons_data.iloc[num_cols_1:]
        
        with open(f"TABLES/capital_projects_table/{cons_tds}_capital_projects_1.tex", 'w') as file_handle:
            file_handle.write(make_capital_table_block(block_data_1, num_cols_1))
            
        with open(f"TABLES/capital_projects_table/{cons_tds}_capital_projects_2.tex", 'w') as file_handle:
            file_handle.write(make_capital_table_block(block_data_1, num_cols_1))
    
    pass
    

In [50]:
asset_data= load_asset_data()
overview_data = load_overview_data()
for tds in consolidations.keys(): 
    make_capital_table(tds, asset_data, overview_data)

#### Make Staff Table

In [51]:
def load_staff_data(name_dict):
    #Read budgeted staff and formula allocation
    dev_staff = pd.read_csv('DATA/staff_for_table.csv')
    dev_staff.fillna(0,inplace=True)
    
    def find_cons_tds(name, name_dict):
        for key, value in name_dict.items():
            if (name == value['name']) | (name in value['alternates']):
                return key
            
    dev_staff['CONS_TDS'] = dev_staff['Consolidation'].apply(lambda x: find_cons_tds(x, name_dict))
    dev_staff['CONS_NAME'] = dev_staff['CONS_TDS'].apply(lambda x: name_dict[x]['name'] if x is not None else 'NO NAME FOUND')
    #Note: Staff list missing for Armstrong, Ft. Washington, and Williams Plaza, as well as scatter-site third-party-managed consolidations
 
    #Read budgeted staff and actuals
    actuals_data = pd.read_csv('DATA/Staffing_Analysis/DEVHC.csv')
    actuals_data.fillna(0, inplace=True)
    actuals_data = actuals_data[actuals_data['RC Name'].apply(lambda x: "total" not in str(x).lower()) & actuals_data['Department'].apply(lambda x: "total" not in str(x).lower())]
    actuals_data['CONS_TDS'] = actuals_data['RC Name'].apply(lambda x: find_cons_tds(x, name_dict))
    actuals_data['CONS_NAME'] = actuals_data['CONS_TDS'].apply(lambda x: name_dict[x]['name'] if x is not None else 'NO NAME FOUND')

    def convert_neg(x):
        try:
            return int(x)
        except:
            return int('-'+str(x).replace('(','').replace(')',''))

    actuals_data['VARIANCE'] = actuals_data['Unnamed: 5'].apply(lambda x: convert_neg(x))
    actuals_data['ACT'] = actuals_data['13']
    
    table_frame = pd.read_csv('DATA/Table_Keys.csv')
    actuals_keys = pd.read_csv('DATA/Staffing_Analysis/DEVHC_CODES.csv')
    
    actuals_data = actuals_data.merge(actuals_keys, how='left', left_on='CST_NAME', right_on='TITLE_NAME')
    for column in ['Current Modified', 'ACT', 'VARIANCE']:
        actuals_data[column] = actuals_data[column].astype(int)
        

    
    return (dev_staff, actuals_data, table_frame, actuals_keys)

In [52]:
def make_staff_table(cons_tds, dev_staff, actuals_data, table_frame, actuals_keys):
    #Fetching staff data for consolidation
    cons_data = dev_staff.loc[dev_staff['CONS_TDS'] == cons_tds]
    if cons_data.shape[0] == 0:
        return(f'Consolidation {cons_tds} not found in staffing data.')
    #print(cons_tds)
    #print(dev_staff)
    
    # Isolate and process actuals data for consolidation
    try:
        cons_actuals = actuals_data[actuals_data['CONS_TDS'] == cons_tds]
    except:
        print(f'{cons_tds} not found in actuals.')
        return np.NaN
    
    cons_actuals = cons_actuals[['CONS_NAME', 'CONS_TDS', 'Current Modified', 'ACT', 
                                 'CODE_KEY', 'CODE_NAME']].groupby(by='CODE_KEY', as_index=False).agg({'CONS_NAME': 'first',
                                                                                                       'CONS_TDS': 'first',
                                                                                                     'Current Modified':sum,
                                                                                                     'ACT':sum,
                                                                                                     'CODE_NAME':'first'})
    cons_actuals
    cons_actuals.loc['Total']= cons_actuals.sum(numeric_only=True, axis=0)
    cons_actuals.loc['Total','CODE_KEY'] = 11
    cons_actuals.loc['Total','CODE_NAME'] = 'TOT'
    
    for row in cons_actuals.itertuples():
        cons_data[f'{row.CODE_NAME}_ACT'] = row.ACT
    #print(cons_data)
    #Setting up table and transposing data
    cons_table_frame = table_frame
    cons_table_frame['Formula'] = cons_table_frame['FORMULA_KEY'].iloc[:-1].apply(lambda key: cons_data[key].iloc[0])
    cons_table_frame['Budgeted'] = cons_table_frame['BUDG_KEY'].apply(lambda key: cons_data[key].iloc[0])
    cons_table_frame['Actual'] = cons_table_frame['ACTUALS_KEY'].iloc[:-2].apply(lambda key: cons_data[key].iloc[0] if key in cons_data.columns else 0)

    
    #Simplifying table
    cons_table = cons_table_frame[['CHART_LINE', 'Formula', 'Budgeted', 'Actual']]
    #print(cons_table)
    
    #Defining LaTeX table format
    
    def make_staff_table_block(staff_data):
    
        table_template = r'''
        \begin{tabular}{l|c|c|c|}
        \cline{2-4}
                                                                                     & \cellcolor{ccfuschia}{\color[HTML]{FFFFFF} Formula Allocation} & \cellcolor{ccfuschia}{\color[HTML]{FFFFFF} Budgeted} & \cellcolor{ccfuschia}{\color[HTML]{FFFFFF} Actual} \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Employees}                      & %s                                                      & %s                                                                & %s                                                        \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Property Manager}               & %s                                                      & %s                                                                & %s                                                       \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Asst. Property Manager}         & %s                                                      & %s                                                                & %s                                                       \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Secretaries}                    & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Housing Assistants}             & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Superintendent}                 & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Assistant Superintendent}       & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Supervisor of Caretakers (SOC)} & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Supervisor of Grounds (SOG)}    & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Maintenance Workers}            & %s                                                      & %s                                                                & %s                                                       \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Caretakers X}                   & %s                                                      & %s                                                                &                                                       \\ \cline{1-3}
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Caretakers J\tnote{1}}                   &                                                       & %s                                                                &                                                         \\ \cline{1-1} \cline{3-3}
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Caretakers G}                   & \multirow{-2}{*}{%s}                                                      & %s                                     & \multirow{-3}{*}{%s}                           \\ \hline
        \end{tabular}
        
        '''

        values = []

        def extract_data_through_mw(row):
            [values.append(item) for item in [str(int(row['Formula'])), 
                                              str(int(row['Budgeted'])), 
                                              str(int(row['Actual']))]]
            pass

        #Processing through Maintenance Worker
        staff_data.iloc[0:-3].apply(lambda row: extract_data_through_mw(row), axis=1)

        #Processing Caretakers
        values.append(str(int(staff_data.iloc[-3, 1])))
        values.append(str(int(staff_data.iloc[-3, 2])))
        values.append(str(int(staff_data.iloc[-2, 2])))
        values.append(str(int(staff_data.iloc[-2, 1])))
        values.append(str(int(staff_data.iloc[-1, 2])))
        values.append(str(int(staff_data.iloc[-3, 3])))

        return table_template % tuple(values)
    
    #Make and export LaTeX code
    with open(f'TABLES/staff_table/{cons_tds}_staff_table.tex', 'w') as file_handle:
        file_handle.write(make_staff_table_block(cons_table))

In [53]:
staff_data = load_staff_data(consolidations)

for tds in consolidations.keys():
    make_staff_table(tds, *staff_data)

/Users/kyleslugg/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Making and Compiling LaTeX Files

In [54]:
def compile_latex_file(tds, counts=counts):
    #SET UTILITY PATHS HERE
    pdflatex_path = '/usr/local/texlive/2018/bin/x86_64-darwin/pdflatex'
    ghostscript_path = '/usr/local/bin/gs'
    
    if counts[tds]['count'] <= 4:
        with open('REPORT_TEMPLATE/report.tex', 'r') as file_handle:
            text = file_handle.read()
            
        new_text = text.replace('$tds_number$', str(tds))

        with open(f'REPORTS/LaTeX/{tds}_report.tex', 'w') as outfile:
            outfile.write(new_text)
    
    else:
        with open('REPORT_TEMPLATE/report_long.tex', 'r') as file_handle:
            text = file_handle.read()
            
        new_text = text.replace('$tds_number$', str(tds))

        with open(f'REPORTS/LaTeX/{tds}_report.tex', 'w') as outfile:
            outfile.write(new_text)

    subprocess.check_call([pdflatex_path, '-output-directory', 'REPORTS/LaTeX', f'REPORTS/LaTeX/{tds}_report.tex'])
    subprocess.check_call([pdflatex_path, '-output-directory', 'REPORTS/LaTeX', f'REPORTS/LaTeX/{tds}_report.tex'])
    
    #Be sure to install ghostscript (to compress pdfs), or comment out next line. Available via homebrew.
    subprocess.check_call([ghostscript_path, '-sDEVICE=pdfwrite', '-dCompatibilityLevel=1.5', '-dNOPAUSE', '-dQUIET', '-dBATCH', f'-sOutputFile=REPORTS/{tds}_report.pdf', f'REPORTS/LaTeX/{tds}_report.pdf'])
    
    pass

In [78]:
if not os.path.exists('REPORTS'):
    os.makedirs('REPORTS')

if not os.path.exists('REPORTS/LaTeX'):
    os.makedirs('REPORTS/LaTeX')

os.system("cp REPORT_TEMPLATE/content.tex REPORTS/LaTeX")
os.system("cp REPORT_TEMPLATE/preface.tex REPORTS/LaTeX")
os.system("cp REPORT_TEMPLATE/content_long.tex REPORTS/LaTeX")

0

In [97]:
consolidation_list = consolidations.keys()

for tds in consolidation_list:
    try:
        compile_latex_file(tds)
    except:
        pass

filelist = [f for f in os.listdir('REPORTS/LaTeX') if not f.endswith(".tex")]
for f in filelist:
    os.remove(os.path.join('REPORTS/LaTeX', f))